# **多變數線性迴歸（Multiple Variable Linear Regression）**

在本實驗中，你會在先前單一特徵的基礎上，將資料結構與相關函式擴充為支援**多個特徵（features）**。

# **1. 大綱**
- [&nbsp;&nbsp;1.1 目標](#toc_15456_1.1)
- [&nbsp;&nbsp;1.2 工具](#toc_15456_1.2)
- [&nbsp;&nbsp;1.3 符號說明](#toc_15456_1.3)
- [2 問題描述](#toc_15456_2)
- [&nbsp;&nbsp;2.1 範例矩陣 X](#toc_15456_2.1)
- [&nbsp;&nbsp;2.2 參數向量 w 與 b](#toc_15456_2.2)
- [3 多變數模型的預測](#toc_15456_3)
- [&nbsp;&nbsp;3.1 逐元素計算的單一預測](#toc_15456_3.1)
- [&nbsp;&nbsp;3.2 使用向量運算的單一預測](#toc_15456_3.2)
- [4 多變數情況下的成本函式](#toc_15456_4)
- [5 多變數情況下的梯度下降](#toc_15456_5)
- [&nbsp;&nbsp;5.1 多變數情況下的梯度計算](#toc_15456_5.1)
- [&nbsp;&nbsp;5.2 多變數情況下的梯度下降實作](#toc_15456_5.2)
- [6 恭喜](#toc_15456_6)


<a name="toc_15456_1.1"></a>
## 1.1 目標

- 將線性迴歸模型的程式擴充為支援多個特徵

    - 擴充資料結構以支援多個特徵

    - 重新撰寫預測、成本與梯度計算等函式，使其支援多個特徵

    - 使用 NumPy 的 `np.dot` 進行向量化實作，以提升速度並簡化程式


<a name="toc_15456_1.2"></a>
## 1.2 工具
在本實驗中，我們會使用： 
- **NumPy**：常用的科學運算函式庫
- **Matplotlib**：常用的資料視覺化與繪圖函式庫

In [ ]:
import sys, os
import copy, math
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
np.set_printoptions(precision=2)

#region 匯入相關模組

def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None

def import_data_from_github():
    import os, urllib.request, pathlib, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_utils_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/utils"
        else:
            return Path.cwd() / "utils"

    env = detect_env()
    UTILS_DIR = get_utils_dir(env)
    REPO_DIR = "Machine-Learning-Lab"

    os.makedirs(UTILS_DIR, exist_ok=True)

    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    urllib.request.urlretrieve(f"{BASE}/utils/deeplearning.mplstyle", f"{UTILS_DIR}/deeplearning.mplstyle")

repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()
    
os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

plt.style.use('utils/deeplearning.mplstyle')
print("✅ 匯入模組及設定繪圖樣式")

#endregion 匯入相關模組

<a name="toc_15456_1.3"></a>

## 1.3 符號說明（Notation）
下面整理了在多特徵情況下會出現的一些常用符號與對應的 Python 實作名稱：  

<div align="center">

| 一般符號 <img width=70/> <br />  Notation  <img width=70/> | 說明<img width=350/> | 對應的 Python 名稱（若適用） |
| ------------| ------------------------------------------------------------|--|
| $a$ | 純量（scalar），一般非粗體字                                                      | |
| $\mathbf{a}$ | 向量（vector），使用粗體小寫字                                                 | |
| $\mathbf{A}$ | 矩陣（matrix），使用粗體大寫字                                         | |
| **迴歸問題（Regression）** |         |    |
|  $\mathbf{X}$ | 訓練樣本矩陣（每列一個樣本）                  | `X_train` |   
|  $\mathbf{y}$  | 訓練樣本的目標值向量                | `y_train` |
|  $\mathbf{x}^{(i)}$, $y^{(i)}$ | 第 $i$ 筆訓練樣本與其標籤 | `X[i]`, `y[i]`|
| m | 訓練樣本的數量 | `m`|
| n | 每個樣本的特徵數量 | `n`|
|  $\mathbf{w}$  |  權重參數向量（weights）                       | `w`    |
|  $b$           |  偏差（bias）參數                                           | `b`    |     
| $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ | 以參數 $\mathbf{w},b$ 在樣本 $\mathbf{x^{(i)}}$ 上得到的模型輸出：$f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)}+b$  | `f_wb` | 

</div>

<br>

<a name="toc_15456_2"></a>

# **2. 問題描述**

我們會延續**房價預測**這個動機範例。訓練資料集中共有三筆樣本，每筆包含四個特徵（房屋坪數、臥室數、樓層數與屋齡），如下表所示。請注意：與先前實驗不同，這裡的房屋大小單位是「平方英尺（sqft）」，而不是「千平方英尺（1000 sqft）」。這將導致一些問題，我們會在下一個實驗中解決！

<div align="center">

| Size (sqft) | Number of Bedrooms  | Number of floors | Age of  Home | Price (1000s dollars)  |   
| ----------------| ------------------- |----------------- |--------------|-------------- |  
| 2104            | 5                   | 1                | 45           | 460           |  
| 1416            | 3                   | 2                | 40           | 232           |  
| 852             | 2                   | 1                | 35           | 178           |  

</div>

你會根據這些資料建立一個線性迴歸模型，之後即可用它來預測其他房屋的價格，例如：一間 1200 平方英尺、3 間臥室、1 層樓、屋齡 40 年的房子。  

請完成下方程式碼 cell 來建立 `X_train` 與 `y_train` 變數。

In [ ]:
X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

<a name="toc_15456_2.1"></a>
## 2.1 範例矩陣 X
與上表類似，訓練樣本會存放在 NumPy 矩陣 `X_train` 中。矩陣的每一列代表一筆訓練樣本。當你有 $m$ 筆訓練樣本（在本例中 $m=3$），且每筆樣本有 $n$ 個特徵（本例中 $n=4$），則 $\mathbf{X}$ 是一個維度為 $(m, n)$ 的矩陣（m 列、n 行）。

$$\mathbf{X} = 
\begin{pmatrix}
 x^{(0)}_0 & x^{(0)}_1 & \cdots & x^{(0)}_{n-1} \\ 
 x^{(1)}_0 & x^{(1)}_1 & \cdots & x^{(1)}_{n-1} \\
 \cdots \\
 x^{(m-1)}_0 & x^{(m-1)}_1 & \cdots & x^{(m-1)}_{n-1} 
\end{pmatrix}
$$
符號說明：
- $\mathbf{x}^{(i)}$ 是第 $i$ 筆樣本所對應的向量：$\mathbf{x}^{(i)} = (x^{(i)}_0, x^{(i)}_1, \cdots,x^{(i)}_{n-1})$
- $x^{(i)}_j$ 是第 $i$ 筆樣本中的第 $j$ 個特徵值。括號中的上標表示「第幾筆樣本」，而下標則代表「第幾個特徵」。  

請先將輸入資料印出來觀察。

In [ ]:
# data is stored in numpy array/matrix
print(f"X Shape: {X_train.shape}, X Type:{type(X_train)})")
print(X_train)
print(f"y Shape: {y_train.shape}, y Type:{type(y_train)})")
print(y_train)

<a name="toc_15456_2.2"></a>
## 2.2 參數向量 w 與 b

* $\mathbf{w}$ 是一個包含 $n$ 個元素的向量。
  - 每個元素對應一個特徵的參數。
  - 在本資料集中，$n = 4$。
  - 在概念圖上，我們通常將 $\mathbf{w}$ 畫成一個「欄向量（column vector）」：

$$\mathbf{w} = \begin{pmatrix}
w_0 \\ 
w_1 \\
\cdots\\
w_{n-1}
\end{pmatrix}
$$
* $b$ 則是單一的純量參數（bias）。  

在示範中，我們會先將 $\mathbf{w}$ 與 $b$ 設定為一組「接近最佳解」的初始值。這裡的 $\mathbf{w}$ 是一個一維的 NumPy 向量。

In [ ]:
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])
print(f"w_init shape: {w_init.shape}, b_init type: {type(b_init)}")

<br>

<a name="toc_15456_3"></a>

# **3. 多變數模型的預測**

在多變數線性迴歸中，模型的預測可以寫成下列線性形式：

$$ f_{\mathbf{w},b}(\mathbf{x}) =  w_0x_0 + w_1x_1 +... + w_{n-1}x_{n-1} + b \tag{1}$$
或使用向量記號：
$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b  \tag{2} $$ 
其中 $\cdot$ 表示向量的「內積（dot product）」。

為了示範內積的用法，我們會分別依照式 (1) 與式 (2) 來實作預測函式。

<a name="toc_15456_3.1"></a>
## 3.1 逐元素計算的單一預測
在前面的單變數線性迴歸中，我們的預測方式是：將一個特徵值乘上一個參數，最後再加上偏差參數（bias）。

若要直接把這個作法擴充到多個特徵，一個直覺的作法就是根據上面的式 (1)，對每一個特徵使用迴圈：逐一將「特徵值 × 對應參數」加總起來，最後再加上偏差參數 $b$。

請用迴圈型式完成函式 `predict_single_loop(x, w, b)` 於下方cell中:

In [ ]:
def predict_single_loop(x, w, b): 
    """
    single predict using linear regression
    
    Args:
      x (ndarray): Shape (n,) example with multiple features
      w (ndarray): Shape (n,) model parameters    
      b (scalar):  model parameter     
      
    Returns:
      p (scalar):  prediction
    """
    n = x.shape[0]
    p = 0
    for i in range(n):
        p_i = x[i] * w[i]  
        p = p + p_i         
    p = p + b                
    return p

In [ ]:
# get a row from our training data
x_vec = X_train[0,:]
print(f"x_vec shape {x_vec.shape}, x_vec value: {x_vec}")

# make a prediction
f_wb = predict_single_loop(x_vec, w_init, b_init)
print(f"f_wb shape {f_wb.shape}, prediction: {f_wb}")

請留意 `x_vec` 的形狀：它是一個具有 4 個元素的一維 NumPy 向量，shape 為 `(4,)`。而 `f_wb` 則是一個純量（scalar）。

<a name="toc_15456_3.2"></a>
## 3.2 使用向量運算的單一預測

注意到，上面的式 (1) 可以改寫為式 (2) 所示的向量內積形式，因此我們可以運用向量運算，來加速預測的計算。

回想在 Python/NumPy 實驗中提到的，NumPy 的 `np.dot()` [[官方文件](https://numpy.org/doc/stable/reference/generated/numpy.dot.html)] 可以用來計算向量內積。 

請用向量運算型式來完成函式 `predict(x, w, b)` 與下方cell:

In [ ]:
def predict(x, w, b): 
    """
    single predict using linear regression
    Args:
      x (ndarray): Shape (n,) example with multiple features
      w (ndarray): Shape (n,) model parameters   
      b (scalar):             model parameter 
      
    Returns:
      p (scalar):  prediction
    """
    p = np.dot(x, w) + b     
    return p    

In [ ]:
# get a row from our training data
x_vec = X_train[0,:]
print(f"x_vec shape {x_vec.shape}, x_vec value: {x_vec}")

# make a prediction
f_wb = predict(x_vec,w_init, b_init)
print(f"f_wb shape {f_wb.shape}, prediction: {f_wb}")

可以看到，這裡得到的結果與使用迴圈時完全相同，shape 也一致。

接下來在本課程中，這類運算會盡量改用 `np.dot` 完成。這樣預測就只需要一行程式碼，因此在後續的函式中，多半會直接嵌入這個運算，而不額外呼叫獨立的 `predict` 函式。

<br>

<a name="toc_15456_4"></a>

# **4. 多變數情況下的成本函式**

在多變數線性迴歸中，成本函式 $J(\mathbf{w},b)$ 可寫為：

$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{3}$$ 

其中：

$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{4} $$ 

與之前單一特徵的實驗不同的是，這裡的 $\mathbf{w}$ 與 $\mathbf{x}^{(i)}$ 都是**向量**，用來同時處理多個特徵，而不再是單一純量。

請完成下方函式 `compute_cost(X, y, w, b)` 程式碼，來實作上面 式 (3) 與式 (4)。

注意：請使用一個 for 迴圈，遍歷所有 `m` 筆訓練樣本，逐筆累加成本。

In [ ]:
def compute_cost(X, y, w, b): 
    """
    compute cost
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """
    m = X.shape[0]
    cost = 0.0
    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b           #(n,)(n,) = scalar (see np.dot)
        cost = cost + (f_wb_i - y[i])**2       #scalar
    cost = cost / (2 * m)                      #scalar    
    return cost

In [ ]:
# Compute and display cost using our pre-chosen optimal parameters. 
cost = compute_cost(X_train, y_train, w_init, b_init)
print(f'Cost at optimal w : {cost}')

**預期結果**：Cost at optimal w : 1.5578904045996674e-12

<br>

<a name="toc_15456_5"></a>

# **5. 多變數情況下的梯度下降**

多變數線性迴歸的梯度下降可寫成：

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline\;
& w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{5}  \; & \text{for j = 0..n-1}\newline
&b\ \ = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

其中，$n$ 是特徵數量，各個參數 $w_j$ 與 $b$ 會在每一步同時更新，而偏導數為：  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{6}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{7}
\end{align}
$$
* $m$：資料集中訓練樣本的數量
* $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$：模型在第 $i$ 筆樣本上的預測值，而 $y^{(i)}$ 是對應的目標值（真實標籤）


<a name="toc_15456_5.1"></a>
## 5.1 多變數情況下的梯度計算

請完成下方函式 `compute_gradient(X, y, w, b)` 程式碼，來實作上面 式 (6) 與式 (7) 的梯度計算。

其實有很多種寫法，這邊我們限定採用：

- 外層迴圈：遍歷所有 $m$ 筆樣本

    - 對每一筆樣本，直接計算並累加對 $b$ 的偏導數 $\frac{\partial J(\mathbf{w},b)}{\partial b}$

    - 再在一個內層迴圈中，針對所有 $n$ 個特徵：
        - 計算並累加每個 $w_j$ 對應的偏導數 $\frac{\partial J(\mathbf{w},b)}{\partial w_j}$
   

In [ ]:
def compute_gradient(X, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i] #scalar
        dj_dw = dj_dw + err * X[i]
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_db, dj_dw

下方函式 `compute_gradient_vectorization(X, y, w, b)` 程式碼為全向量化實作上面 式 (6) 與式 (7) 的梯度計算。(供參考)

In [ ]:
def compute_gradient_vectorization(X, y, w, b):
    m = X.shape[0]
    y_pred = np.dot(X, w) + b      # (m,)
    error = y_pred - y             # (m,)
    
    dj_dw = (1/m) * np.dot(X.T, error)  # (n,)
    dj_db = (1/m) * np.sum(error)       # scalar
    return dj_dw, dj_db

In [ ]:
#Compute and display gradient 
tmp_dj_db, tmp_dj_dw = compute_gradient(X_train, y_train, w_init, b_init)
print(f'dj_db at initial w,b: {tmp_dj_db}')
print(f'dj_dw at initial w,b: \n {tmp_dj_dw}')

**預期結果**：   
 dj_db at initial w,b: -1.6739251122999121e-06  
 dj_dw at initial w,b:   
 [-2.73e-03 -6.27e-06 -2.22e-06 -6.92e-05]    

<a name="toc_15456_5.2"></a>
## 5.2 多變數情況下的梯度下降實作
下面這個函式實作了上面式 (5) 所描述的梯度下降過程。

In [ ]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    """
    Performs batch gradient descent to learn w and b. Updates w and b by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters  
      b_in (scalar)       : initial model parameter
      cost_function       : function to compute cost
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : Updated values of parameters 
      b (scalar)       : Updated value of parameter 
      """
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db,dj_dw = gradient_function(X, y, w, b)   ##None

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               ##None
        b = b - alpha * dj_db               ##None
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(X, y, w, b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
        
    return w, b, J_history #return final w,b and J history for graphing

在下一個程式碼 cell 中，你會實際測試這個梯度下降實作。 

In [ ]:
# initialize parameters
initial_w = np.zeros_like(w_init)
initial_b = 0.
# some gradient descent settings
iterations = 1000
alpha = 5.0e-7
# run gradient descent 
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b,
                                                    compute_cost, compute_gradient, 
                                                    alpha, iterations)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")
m,_ = X_train.shape
for i in range(m):
    print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.2f}, target value: {y_train[i]}")

**預期結果**：    
 b,w found by gradient descent: -0.00,[ 0.2   0.   -0.01 -0.07]   
 prediction: 426.19, target value: 460  
 prediction: 286.17, target value: 232  
 prediction: 171.47, target value: 178    

In [ ]:
# plot cost versus iteration  
fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(12, 4))
ax1.plot(J_hist)
ax2.plot(100 + np.arange(len(J_hist[100:])), J_hist[100:])
ax1.set_title("Cost vs. iteration");  ax2.set_title("Cost vs. iteration (tail)")
ax1.set_ylabel('Cost')             ;  ax2.set_ylabel('Cost') 
ax1.set_xlabel('iteration step')   ;  ax2.set_xlabel('iteration step') 
plt.show()

*這樣的結果並不是特別令人振奮*！成本仍然在持續下降，而且目前的預測也還不夠準確。下一個實驗會探討如何改進這種情況。

<br>


<a name="toc_15456_6"></a>

# **6. 恭喜！**

在本實驗中，你已經：
- 重新整理並擴充了線性迴歸所需的各種函式，讓它們可以處理多個特徵。
- 使用 NumPy 的 `np.dot` 來將實作向量化，以提升運算效率與簡潔度。